# Recap on Decision Trees

```
Authors: Alexandre Gramfort
         Thomas Moreau
         Mathurin Massias
```

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Decision trees encode a series of "if" and "else" choices, similar to how a person might make a decision.
However, which questions to ask, and how to proceed for each answer is entirely learned from the data.

For example, if you wanted to create a guide to identifying an animal found in nature, you
might ask the following series of questions:

- Is the animal **bigger or smaller than a meter long**?
    + *bigger*: does the animal **have horns**?
        - *yes*: are the horns longer than ten centimeters?
        - *no*: is the animal wearing a collar
    + *smaller*: does the animal **have two or four legs**?
        - *two*: does the animal have wings?
        - *four*: does the animal have a bushy tail?

and so on.  This binary splitting of questions is the essence of a decision tree.

One of the main **benefit of tree-based models** is that they require **little preprocessing of the data**.
They can be used for **regression and classification**, work with **variables of different types** (continuous and categorical) and are **invariant to scaling of the features**.

Another benefit is that tree-based models are what is called **"nonparametric"**, which means they don't have a fix set of parameters to learn. Instead, **a tree model can become more and more flexible**, if given more data.
In other words, the **number of free parameters grows with the number of samples** and is not fixed, as for example in linear models.

## Decision Tree for Classification

### Generate a simple dataset

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(
    centers=[[0, 0], [2, 2]], random_state=61526, n_samples=150
)

First, let's look at the generated data

In [ ]:
classes = np.unique(y)
print(f"The class labels are: {classes}")

In [ ]:
_, ax = plt.subplots()
for klass, color in zip(classes, ["tab:orange", "tab:blue"]):
    mask_sample_klass = y == klass
    ax.scatter(
        X[mask_sample_klass, 0], X[mask_sample_klass, 1],
        color=color, label=klass,
        edgecolor="black",
    )
plt.axis("square")
plt.legend()
plt.xlabel("Feature #0")
_ = plt.ylabel("Feature #1")

We will create a function to create this scatter plot by passing 2 variables: `data` and `labels`.

In [ ]:
def plot_data(data, labels, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    classes = np.unique(labels)
    for klass, color in zip(classes, ["tab:orange", "tab:blue"]):
        mask_sample_klass = labels == klass
        ax.scatter(
            *data[mask_sample_klass].T,
            color=color, label=klass,
            edgecolor="black",
        )
    sns.despine()
    ax.axis("square")
    plt.legend()
    plt.xlabel("Feature #0")
    plt.ylabel("Feature #1")
    return ax

In [ ]:
plot_data(X, y);

### Train a decision tree classifier

We can learn a set of binary rules using a portion of the data. Using the rules learned, we will predict on the testing data.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42,
)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train)
pred = clf.predict(X_test)
pred

We can plot the decision boundaries found using the training data.

In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay
from matplotlib import cm 
import matplotlib 
from matplotlib.colors import ListedColormap


top = matplotlib.colormaps['Oranges_r']
bottom = matplotlib.colormaps['Blues']
newcolors = np.vstack((top(np.linspace(0, 1, 128)),
                       bottom(np.linspace(0, 1, 128))))
cmap = ListedColormap(newcolors, name='OrangeBlue')

def plot_tree_boundary(depth=5):
    clf = DecisionTreeClassifier(max_depth=depth).fit(X_train, y_train)
    display = DecisionBoundaryDisplay.from_estimator(
        clf, X, alpha=0.5, response_method="predict_proba", cmap=cmap
    )
    _ = plot_data(X_train, y_train, ax=display.ax_)

plot_tree_boundary()

Similarly, we get the following classification on the testing set.

In the interactive plot below, the regions are assigned blue and orange colors to indicate the predicted class for that region. The shade of the color indicates the predicted probability for that class (darker = higher probability), while white regions indicate an equal predicted probability for either class.

In [ ]:
def plot_tree_interactive():
    from ipywidgets import interactive, IntSlider
    slider = IntSlider(min=1, max=8, step=1, value=1)
    return interactive(plot_tree_boundary, depth=slider)

plot_tree_interactive()


### Learning the splitting rule in a decision tree for binary classification

In this section, we will go slightly more into details regarding how a tree selects the best split. First, instead of using synthetic data, we will use a real dataset this time.

In [ ]:
dataset = pd.read_csv("penguins.csv")
dataset = dataset.dropna(subset=["Body Mass (g)"])
dataset.head()

We will build a decision tree to classify the penguin species using their body mass as a feature. To simplify the problem will focus only the Adelie and Gentoo species.

In [ ]:
# Only select the column of interest
dataset = dataset[["Body Mass (g)", "Species"]]
# Make the species name more readable
dataset["Species"] = dataset["Species"].apply(lambda x: x.split()[0])
# Only select the Adelie and Gentoo penguins
dataset = dataset.set_index("Species").loc[["Adelie", "Gentoo"], :]
# Sort all penguins by their body mass
dataset = dataset.sort_values(by="Body Mass (g)")
# Convert the dataframe (2D) to a series (1D)
dataset = dataset.squeeze()
dataset

We will first look at the body mass distribution for each specie.

In [ ]:
_, ax = plt.subplots()
dataset.groupby("Species").plot.hist(ax=ax, alpha=0.7, legend=True, density=True)
ax.set_ylabel("Probability")

When we build a tree, we want to find splits, one at the time, such that we partition the data in way that classes as "unmixed" (homogeneous) as possible in the resulting partition. Let's make a first completely random split to highlight the principle.

In [ ]:
# create a random state such we all have the same results
rng = np.random.RandomState(42)
random_idx = rng.choice(dataset.size)

ax = sns.swarmplot(x=dataset.values, y=[""] * len(dataset),
                   hue=dataset.index)
ax.set_xlabel(dataset.name)
ax.set_title(f"Body mass threshold: {dataset.iloc[random_idx]} grams")
_ = ax.vlines(dataset.iloc[random_idx], -1, 1, color="red", linestyle="--")

We seek for a partition in two sets for which 
- the samples in each set are as much as possible from a single class
- **each set contains as many samples as possible**. 

In a decision tree, this is measured by a **numerical criterion** which assesses the quality of a split. The **entropy** is one of the statistics which can describe the class mixity in a partition. Let's compute the entropy for the full dataset, the set on the left of the threshold and the set on the right of the split.

Let's first look at the entropy function:

In [ ]:
from scipy.stats import entropy

ps = np.linspace(0, 1, 100)
ents = [entropy([p, 1-p]) for p in ps]
plt.figure(figsize=(6, 3))
plt.plot(ps, ents)
plt.ylabel('Entropy of cell')
plt.xlabel('Proportion of class 1 in cell');

As you can see the entropy is 0 when you have no mixing between classes and it's maximal when you have 50% of each class.

In [ ]:
dataset.index.value_counts(normalize=True)  # proportions in whole dataset

In [ ]:
parent_entropy = entropy(
    dataset.index.value_counts(normalize=True)
)
parent_entropy

In [ ]:
left_entropy = entropy(
    dataset[:random_idx].index.value_counts(normalize=True)
)
left_entropy

In [ ]:
right_entropy = entropy(
    dataset[random_idx:].index.value_counts(normalize=True)
)
right_entropy

We can see the quality of the split by combining the entropies. This is known as the **information gain**.

In [ ]:
parent_entropy - (left_entropy + right_entropy)

To favor splits that do not create small regions that are too small, we normalize the entropies by the number of samples in each resulting cell.

In [ ]:
def information_gain(labels_parent, labels_left, labels_right):
    # compute the entropies
    entropy_parent = entropy(labels_parent.value_counts(normalize=True))
    entropy_left = entropy(labels_left.value_counts(normalize=True))
    entropy_right = entropy(labels_right.value_counts(normalize=True))

    n_samples_parent = labels_parent.size
    n_samples_left = labels_left.size
    n_samples_right = labels_right.size

    # normalize with the number of samples
    normalized_entropy_left = ((n_samples_left / n_samples_parent) * 
                               entropy_left)
    normalized_entropy_right = ((n_samples_right / n_samples_parent) *
                                entropy_right)

    return (entropy_parent -
            normalized_entropy_left - normalized_entropy_right)

In [ ]:
information_gain(
    dataset.index,
    dataset[:random_idx].index,
    dataset[random_idx:].index
)

So, we can compute the information gain for all possible body mass thresholds.

In [ ]:
all_information_gain = pd.Series(
    [information_gain(dataset.index, dataset[:idx].index, dataset[idx:].index)
     for idx in range(dataset.size)],
    index=dataset,
)

ax = all_information_gain.plot(color="red", label="Information gain")
ax = sns.swarmplot(x=dataset.values, y=[""] * len(dataset), hue=dataset.index)
ax.invert_yaxis()

The maximum of the information gain corresponds to the split which best partitions our data (according to this criterion). We can check the corresponding body mass threshold.

In [ ]:
all_information_gain.idxmax()

In [ ]:
ax = all_information_gain.plot(color="red", label="Information gain")
ax = sns.swarmplot(x=dataset.values, y=[""] * len(dataset), hue=dataset.index)
ax.invert_yaxis()
ax.vlines(
    all_information_gain.idxmax(), -1, 1,
    color="k", linestyle="--"
);

**Question**: what is the cost of finding the best split in the above example, if there are $n$ points?

Note: the search for the best split along one feature can be performed in linear time if the data is sorted, so the cost for one split is $\mathcal{O}(n \log n)$.


## Finding the best split in dimension 2

Let's now see how trees perform splits in dimension $> 1$.

On a toy 2D example, we can visualize the information gain for splits on the first or second feature

In [ ]:
X, y = make_blobs(centers=[[0, 0], [0, 1.2], [2, 0]], cluster_std=0.5, random_state=0)
y[y==2] = 1  # otherwise there are 3 blobs hence 3 classes

fig = plt.figure(figsize=(10, 10))
ax = fig.add_gridspec(top=0.4, right=0.4).subplots()
plot_data(X, y, ax)

ax0 = ax.inset_axes([0, 1.05, 1, 0.25], sharex=ax)
ax1 = ax.inset_axes([1.05, 0, 0.25, 1], sharey=ax)

for feat, ax in zip([0, 1], [ax0, ax1]):
    idx = np.argsort(X[:, feat])
    info_gain = [information_gain(
        pd.Series(y), pd.Series(y[idx[:i]]), pd.Series(y[idx[i:]])) for i in range(len(y))]
    if feat == 0:
        ax.plot(X[idx, feat], info_gain)
        ax.tick_params(axis="x", labelbottom=False)
        ax.set_ylabel("info gain")
        ax.set_ylim(0, 0.2)
    else:
        ax.plot(info_gain, X[idx, feat])
        ax.tick_params(axis="y", labelleft=False)
        ax.set_xlabel("info gain")
        ax.set_xlim(0, 0.2)

## Decision Tree for Regression

The model is exactly the same: a recursive partitioning of the whole space into rectangular cells, with constant prediction per cell (e.g. the average of targets falling in that cell). 
The tree is built with iterative splits performed in a greedy way in order to maximally decrease some quantity, e.g. the RMSE/variance in the two resulting cells.

Here we use a one dimensional example, so the cells are just segments (and two half lines).

In [ ]:
rnd = np.random.RandomState(42)
x = np.linspace(-3, 3, 100)
y_no_noise = np.sin(4 * x) + x
y = y_no_noise + rnd.normal(size=len(x))
X = x.reshape(-1, 1)

plt.figure()
plt.xlabel('Feature X')
plt.ylabel('Target y')
_ = plt.scatter(X, y)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
reg = DecisionTreeRegressor(max_depth=2)
reg.fit(X_train, y_train)

In [ ]:
plt.figure()
plt.plot(x, reg.predict(x[:, None]), color='tab:blue', label="prediction")
plt.plot(X_train.squeeze(), y_train, 'C7.', label="training data")
_ = plt.legend(loc="best")

A single decision tree allows us to estimate the signal in a non-parametric way,
but clearly has some issues.  In some regions, the model shows high bias and
under-fits the data.
(seen in the long flat lines which don't follow the contours of the data),
while in other regions the model shows high variance and over-fits the data
(reflected in the narrow spikes which are influenced by noise in single points).

## Overfitting in trees

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>
      Take the above example and repeat the training/testing by changing the depth of the tree.
      </li>
    </ul>
</div>